In [3]:
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

# Data

data_link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/'

In [6]:
data_path = "data/winequality-red.csv"
data = pd.read_csv(data_path, sep=";")

data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
435,12.3,0.39,0.63,2.3,0.091,6.0,18.0,1.00040,3.16,0.49,9.5,5
246,7.1,0.68,0.07,1.9,0.075,16.0,51.0,0.99685,3.38,0.52,9.5,5
757,8.1,0.87,0.00,2.2,0.084,10.0,31.0,0.99656,3.25,0.50,9.8,5
899,8.3,1.02,0.02,3.4,0.084,6.0,11.0,0.99892,3.48,0.49,11.0,3
70,7.7,0.63,0.08,1.9,0.076,15.0,27.0,0.99670,3.32,0.54,9.5,6


# 1. Tracking experiments

To start MLflow server,

`mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host localhost --port 5000`

In [8]:
remote_server_uri = "http://localhost:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [9]:
mlflow.tracking.get_tracking_uri()

'http://localhost:5000'

In [10]:
# Set an experiment like in git creating branch or directly go to it if exists
exp_name = "Elasticnet_wine"
mlflow.set_experiment(exp_name)

2022/08/05 01:48:38 INFO mlflow.tracking.fluent: Experiment with name 'Elasticnet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='Elasticnet_wine', tags={}>

# 2. Modelling

In [11]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def load_data(data_path):
    data = pd.read_csv(data_path, sep=";")
    
    # Split data to train, test (0.75,0.25) split
    train, test = train_test_split(data)
    
    # The predicted column is 'quality' which is a scalar from [3,9]
    train_x = train.drop(['quality'], axis=1)
    test_x = test.drop(['quality'], axis=1)
    train_y = train['quality']
    test_y = test['quality']
    
    return train_x, train_y, test_x, test_y

In [23]:
def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameter
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    # Read the wine quality csv file (make sure it is running from root directory)
    data_path = "data/winequality-red.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)
    
    # Useful for multiple runs (only doing one run here)
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        # Evaluate metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        # Print out the metrics
        print("ElasticNet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print(" RMSE: %s" % rmse)
        print(" MAE: %s" % mae)
        print(" R2: %s" % r2)
        
        # Log parameter, metrics, and model to MLFlow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})        
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [13]:
# could also do to track values over multiple steps
# with mlflow.start_run():
#     for epoch in range(0, 3):
#         mlflow.log_metric(key="quality", value=2*epoch, step=epoch)

In [14]:
train(0.5, 0.5)

ElasticNet model (alpha=0.500000, l1_ratio=0.500000):
 RMSE: 0.793164022927685
 MAE: 0.6271946374319586
 R2: 0.10862644997792625
Save to: mlruns/1/804d002393cc4502808bb7c70d105e55/artifacts


In [15]:
train(0.2, 0.2)

ElasticNet model (alpha=0.200000, l1_ratio=0.200000):
 RMSE: 0.7336400911821402
 MAE: 0.5643841279275427
 R2: 0.2373946606358417
Save to: mlruns/1/3ca04bf6e3d44452855f8d1cf0e8124a/artifacts


In [16]:
train(0.1, 0.1)

ElasticNet model (alpha=0.100000, l1_ratio=0.100000):
 RMSE: 0.7128829045893679
 MAE: 0.5462202174984664
 R2: 0.2799376066653345
Save to: mlruns/1/2f9e209a3e4a493896f0ca84499842f4/artifacts


**Note**:

* Getting some issues with running `mlflow ui` command. Getting error.
* So directly opened link, which seems to work